In [1]:
import numpy as np
import random
import os
import time
import torch 
from torch import tensor
from torch.autograd import Variable
from copy import deepcopy
from random import shuffle
from tensorboardX import SummaryWriter

In [2]:
from lib.turn_program_into_file import turn_program_into_file
from game.wrappers.game_container import GameContainer

In [3]:
cpus = 4
player_size = 8
game = GameContainer(128, 128)
# Each run taks 2.6 sec per cpu, 38089 = a 24h run.
run_limit = 10
save_cycle = 2
output_dir = "./output/pyramid/"
log_path = './output/logs/'
run = int(time.time())
save_dir = output_dir + str(run) + '/'
writer = SummaryWriter()

In [4]:
# king_hill_ids = list(range(0, cpus))
os.makedirs(os.path.join(output_dir, str(run)))

In [5]:
# -*- coding: utf-8 -*-
import torch

top = 2

# Use the nn package to define our model and loss function.
p1 = torch.nn.Sequential(
    torch.nn.Linear(top**6, top**5),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**5, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**3),
    torch.nn.ReLU(),
    torch.nn.Linear(top**3, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**5),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**5, top**6),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**6, top**6),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**6, top**6),
    torch.nn.ReLU(),
    torch.nn.Linear(top**6, top**6),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**6, top**6),
    torch.nn.ReLU(),
    torch.nn.Linear(top**6, top**6),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**6, top**6),
    torch.nn.Sigmoid(),
)


p2 = torch.nn.Sequential(
    torch.nn.Linear(top**6, top**5),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**5, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**3),
    torch.nn.ReLU(),
    torch.nn.Linear(top**3, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**5),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**5, top**6),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**6, top**6),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**6, top**6),
    torch.nn.Sigmoid(),
)


loss_fn = torch.nn.BCELoss()


def return_result(game_result, number):
    result = 0
    
    for i in game_result:
        if i == number:
            result += 1
        
    return result

def create_player(game_board_pt, player_size):
    game_board_np = deepcopy(game_board_pt.detach().numpy())
    
    for i in range(len(game_board_np)):
        game_board_np[i] = 1.0 if game_board_np[i] > 0.5 else 0.0
    
    game_board_np = np.array(game_board_np, dtype=np.int)
    
    return game_board_np


def run_game(home, away, rounds, player_size):
    home = np.reshape(home, (player_size, player_size))
    away = np.reshape(away, (player_size, player_size))
    
    game.add_players(home, away)
    game.launch(rounds)
    final_board = game.gol.grid()
    final_board = np.array(final_board).flatten()
    
    return return_result(final_board, 1)

def create_loss_board(home, away, rounds, player_size, check_order):
    org_score = run_game(home, away, rounds, player_size)
    loss_board = deepcopy(home)
    tile_board = deepcopy(home)
    top_score = org_score
    
    for i in check_order:
        tile_board[i] = 0.0 if home[i] == 1.0 else 1.0
        tile_result = run_game(tile_board, away, rounds, player_size)
        if tile_result >= top_score:
            loss_board[i] = tile_board[i]
            top_score = tile_result
        else:
            tile_board = deepcopy(loss_board)
    if top_score == org_score:
        shuffle(check_order)
    
    return torch.tensor(loss_board, dtype=torch.float), org_score, top_score, check_order

learning_rate = 0.0002
check_order_p1 = list(range(0, player_size**2))
check_order_p2 = list(range(0, player_size**2))
optimizer_p1 = torch.optim.Adam(p1.parameters(), lr=learning_rate)
optimizer_p2 = torch.optim.Adam(p2.parameters(), lr=learning_rate)

start_p1 = Variable(torch.tensor(torch.rand(player_size * player_size), requires_grad=True))
start_p2 = Variable(torch.tensor(torch.rand(player_size * player_size), requires_grad=True))

for t in range(10000):
    
    p1_board = p1(start_p1)
    p2_board = p2(start_p2)

    with torch.no_grad():
        p1_board_np = create_player(p1_board.clone(), player_size)
        p2_board_np = create_player(p2_board.clone(), player_size)

        p1_loss_board, p1_score, p1_top, check_order_p1 = create_loss_board(p1_board_np, p2_board_np, 10, player_size, check_order_p1)
        p2_loss_board, p2_score, p2_top, check_order_p2 = create_loss_board(p2_board_np, p1_board_np, 10, player_size, check_order_p2)

    loss_p1 = tensor((1 - (p1_score/p1_top))) * loss_fn(p1_board, p1_loss_board)
    loss_p1.backward()

    
    optimizer_p1.step()
    
    loss_p2 = tensor((1 - (p2_score/p2_top))) * loss_fn(p2_board, p2_loss_board)
    loss_p2.backward()
    optimizer_p2.step()
    
    optimizer_p1.zero_grad()
    optimizer_p2.zero_grad()
    
    start_p1 = Variable(torch.tensor(p1_loss_board, requires_grad=True))
    start_p2 = Variable(torch.tensor(p2_loss_board, requires_grad=True))
    
    print(loss_p1, p1_score, p1_top)
    print(loss_p2, p2_score, p2_top)
    
    writer.add_scalars('deep/loss', {'loss_p1': loss_p1, 'loss_p2': loss_p2}, t)
    writer.add_scalars('deep/top', {'p1_top': p1_top, 'p2_top': p2_top}, t)
    writer.add_scalars('deep/score', {'p1_score': p1_score, 'p2_score': p2_score}, t)


    if t % 100 == 0:
        print("Saving players:")
        print(p1_board, p2_board)
#         turn_program_into_file(np.reshape(p1_board_np, (8,8)), save_dir + "p1-cycle-" + str(t) + ".rle", "RKH", "EW", "")
#         turn_program_into_file(np.reshape(p2_board_np, (8,8)), save_dir + "p2-cycle-" + str(t) + ".rle", "RKH", "EW", "")

tensor(0.2763) 30 51
tensor(0.5245) 12 57
Saving players:
tensor([ 0.5130,  0.4823,  0.5026,  0.5137,  0.4828,  0.4918,  0.4997,
         0.5285,  0.5177,  0.5246,  0.5300,  0.4812,  0.5115,  0.4785,
         0.4781,  0.4824,  0.4977,  0.5263,  0.4815,  0.4931,  0.4750,
         0.4820,  0.4896,  0.5123,  0.5136,  0.4843,  0.5084,  0.4871,
         0.5053,  0.5211,  0.4757,  0.5150,  0.5060,  0.5220,  0.5315,
         0.4682,  0.4875,  0.4818,  0.4649,  0.5253,  0.4981,  0.5235,
         0.4910,  0.4770,  0.4945,  0.5100,  0.4989,  0.5157,  0.4939,
         0.4961,  0.4927,  0.4758,  0.5024,  0.4765,  0.5029,  0.5011,
         0.5215,  0.4925,  0.4734,  0.4880,  0.4804,  0.4964,  0.4949,
         0.4795]) tensor([ 0.5034,  0.4894,  0.4830,  0.4889,  0.5158,  0.4750,  0.4705,
         0.4617,  0.4741,  0.5352,  0.5062,  0.4928,  0.4673,  0.5155,
         0.4791,  0.4944,  0.5322,  0.5215,  0.5017,  0.4754,  0.4788,
         0.4927,  0.4602,  0.4761,  0.5211,  0.5215,  0.4873,  0.5155,
 

tensor(0.2842) 22 49
tensor(0.3029) 29 61
tensor(0.2721) 22 49
tensor(0.1690) 30 43
tensor(0.3691) 16 62
tensor(0.1693) 30 43
tensor(0.2372) 22 45
tensor(0.4834) 10 61
tensor(0.1874) 43 70
tensor(0.1710) 30 43
tensor(0.4257) 14 70
tensor(0.1633) 30 43
tensor(0.2759) 19 50
tensor(0.4662) 8 42
tensor(0.2169) 31 59
tensor(0.2533) 30 50
tensor(0.2760) 27 62
tensor(0.2755) 30 61
tensor(0.3043) 28 70
tensor(0.1566) 30 43
tensor(0.2433) 22 49
tensor(0.1635) 30 43
tensor(0.1466) 33 54
tensor(0.4166) 14 53
tensor(0.1773) 43 70
tensor(0.1692) 30 43
tensor(0.1759) 43 70
tensor(0.1663) 30 43
tensor(0.1640) 43 70
tensor(0.1684) 30 43
tensor(0.1910) 22 49
tensor(0.1537) 30 43
tensor(0.1775) 43 70
tensor(0.1626) 41 59
tensor(0.1348) 33 54
tensor(0.1618) 30 43
tensor(0.1626) 43 70
tensor(0.1351) 41 55
tensor(0.1643) 43 70
tensor(0.2693) 30 61
tensor(0.1636) 43 70
tensor(0.3125) 22 48
tensor(0.1922) 22 45
tensor(0.1271) 41 53
tensor(0.1763) 43 70
tensor(0.2491) 30 61
tensor(0.1426) 33 68
tensor(0.1548)

tensor(0.1443) 22 50
tensor(1.00000e-03 *
       4.6388) 43 44
tensor(0.2029) 19 56
tensor(1.00000e-02 *
       5.9211) 43 51
tensor(1.00000e-02 *
       5.1080) 38 49
tensor(0.2540) 24 54
tensor(0.2109) 19 56
tensor(1.00000e-02 *
       5.4520) 43 51
tensor(0.2014) 19 56
tensor(1.00000e-03 *
       5.3037) 43 44
tensor(0.1784) 19 56
tensor(1.00000e-03 *
       4.8352) 43 44
tensor(0.1793) 19 56
tensor(1.00000e-02 *
       5.3437) 43 51
tensor(0.1799) 19 56
tensor(0.1775) 26 56
tensor(0.1845) 19 56
tensor(1.00000e-02 *
       2.2360) 43 46
tensor(0.1004) 33 54
tensor(1.00000e-02 *
       1.5173) 43 46
tensor(1.00000e-02 *
       4.1981) 38 49
tensor(0.1712) 27 56
tensor(1.00000e-02 *
       6.1421) 41 55
tensor(1.00000e-02 *
       2.6752) 43 47
tensor(1.00000e-02 *
       6.8683) 35 49
tensor(0.1830) 26 56
tensor(0.1831) 19 56
tensor(1.00000e-02 *
       8.4214) 37 53
tensor(0.1848) 19 56
tensor(1.00000e-02 *
       6.1919) 43 51
tensor(0.1959) 19 56
tensor(0.1755) 27 56
tensor(0.1694

tensor(0.1151) 22 56
tensor(0.2622) 24 49
tensor(1.00000e-02 *
       9.0560) 25 51
tensor(0.1180) 43 63
tensor(1.00000e-02 *
       9.5940) 25 51
tensor(0.1038) 43 63
tensor(1.00000e-02 *
       2.0306) 50 55
tensor(0.1068) 29 59
tensor(0.1910) 26 53
tensor(0.1394) 29 57
tensor(1.00000e-02 *
       3.4099) 43 56
tensor(0.1012) 43 63
tensor(0.2543) 16 52
tensor(1.00000e-02 *
       3.3727) 43 56
tensor(1.00000e-02 *
       5.2681) 43 63
tensor(1.00000e-02 *
       9.1603) 43 65
tensor(0.2083) 17 60
tensor(0.2673) 24 48
tensor(0.1891) 34 56
tensor(1.00000e-02 *
       7.7490) 42 54
tensor(1.00000e-02 *
       1.5190) 50 55
tensor(0.1149) 19 39
tensor(0.1488) 25 58
tensor(0.1124) 29 53
tensor(1.00000e-02 *
       1.4359) 50 55
tensor(0.2532) 19 59
tensor(1.00000e-02 *
       9.4965) 22 56
tensor(0.1332) 29 57
tensor(1.00000e-02 *
       1.4611) 50 55
tensor(1.00000e-02 *
       3.8023) 43 56
tensor(1.00000e-02 *
       2.6913) 46 51
tensor(1.00000e-02 *
       5.1167) 47 58
tensor(0.3470

tensor(0.2363) 14 56
tensor(1.00000e-02 *
       4.2845) 43 53
tensor(0.3372) 11 51
tensor(1.00000e-02 *
       5.1301) 43 56
tensor(1.00000e-02 *
       2.5604) 51 63
tensor(1.00000e-02 *
       7.7163) 43 63
tensor(1.00000e-02 *
       1.6743) 50 55
tensor(1.00000e-02 *
       7.3803) 43 63
tensor(1.00000e-02 *
       4.7811) 43 63
tensor(1.00000e-02 *
       4.7977) 43 54
tensor(1.00000e-02 *
       4.5761) 43 63
tensor(1.00000e-02 *
       5.1559) 43 54
tensor(1.00000e-02 *
       2.7663) 51 63
tensor(1.00000e-02 *
       7.6395) 43 63
tensor(1.00000e-02 *
       2.6002) 51 63
tensor(1.00000e-02 *
       7.8562) 43 63
tensor(1.00000e-02 *
       2.9599) 51 63
tensor(1.00000e-02 *
       6.8363) 47 64
tensor(1.00000e-02 *
       5.2277) 43 63
tensor(1.00000e-02 *
       3.1264) 43 53
tensor(1.00000e-02 *
       2.6961) 51 63
tensor(1.00000e-02 *
       7.1949) 43 63
tensor(1.00000e-02 *
       2.2144) 51 63
tensor(1.00000e-02 *
       7.6759) 43 63
tensor(1.00000e-02 *
       5.6715

...

tensor(1.00000e-02 *
       2.3141) 55 67
tensor(1.00000e-03 *
       6.3655) 65 71
tensor(1.00000e-02 *
       2.2328) 55 67
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.5777) 55 67
tensor(1.00000e-03 *
       7.3080) 65 71
tensor(1.00000e-02 *
       2.6892) 55 67
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.3068) 55 67
tensor(1.00000e-03 *
       2.9871) 65 71
tensor(1.00000e-02 *
       2.2130) 55 67
tensor(1.00000e-03 *
       3.1752) 65 71
tensor(1.00000e-02 *
       3.1143) 55 67
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.9742) 55 67
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.3682) 55 67
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.1767) 55 67
tensor(0.) 71 71
tensor(1.00000e-02 *
       1.9152) 55 67
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.2186) 55 67
tensor(1.00000e-02 *
       1.0396) 65 71
tensor(0.1388) 41 66
tensor(1.00000e-02 *
       1.2329) 65 71
tensor(1.00000e-02 *
       2.1204) 55 67
tensor(1.00000e-02 *
       1.0643) 65 71
tensor(1.00000e-02 *

tensor(1.00000e-03 *
       5.0099) 62 67
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.2611) 67 74
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.0673) 67 74
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.2061) 67 74
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.1659) 67 74
tensor(1.00000e-02 *
       1.2724) 58 71
tensor(1.00000e-02 *
       2.4309) 67 74
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.1771) 67 74
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.1125) 67 74
tensor(1.00000e-03 *
       2.1874) 65 71
tensor(1.00000e-02 *
       2.0275) 67 74
tensor(0.) 71 71
tensor(1.00000e-02 *
       1.7998) 67 74
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.0996) 67 74
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.0133) 67 74
tensor(0.) 71 71
tensor(1.00000e-02 *
       1.9481) 67 74
tensor(1.00000e-02 *
       1.3905) 58 71
tensor(1.00000e-02 *
       1.8178) 67 74
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.0918) 67 74
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.0472) 67 7

tensor(1.00000e-02 *
       7.8210) 41 58
tensor(0.) 71 71
tensor(1.00000e-02 *
       5.5572) 47 58
tensor(0.) 71 71
tensor(1.00000e-02 *
       8.5217) 41 58
tensor(0.) 71 71
tensor(1.00000e-02 *
       8.0337) 41 58
tensor(1.00000e-03 *
       3.2180) 65 71
tensor(1.00000e-02 *
       8.7238) 41 58
tensor(0.) 71 71
tensor(1.00000e-02 *
       8.8014) 41 58
tensor(0.) 71 71
tensor(1.00000e-02 *
       7.6441) 41 58
tensor(0.) 71 71
tensor(1.00000e-02 *
       8.4802) 41 58
tensor(0.) 71 71
tensor(1.00000e-02 *
       1.5570) 67 74
tensor(0.) 71 71
tensor(1.00000e-02 *
       8.0364) 41 58
tensor(0.) 71 71
tensor(1.00000e-02 *
       7.4307) 41 58
tensor(0.) 71 71
tensor(1.00000e-02 *
       7.7140) 41 58
tensor(0.) 71 71
tensor(1.00000e-02 *
       8.1376) 41 58
tensor(0.) 71 71
tensor(1.00000e-02 *
       1.6742) 67 74
tensor(0.) 71 71
tensor(1.00000e-02 *
       5.3287) 47 58
tensor(1.00000e-03 *
       3.1577) 65 71
tensor(1.00000e-02 *
       8.0932) 41 58
tensor(0.) 71 71
tensor

tensor(1.00000e-02 *
       6.0853) 48 66
tensor(0.) 71 71
tensor(1.00000e-02 *
       7.3073) 62 77
tensor(0.) 71 71
tensor(1.00000e-02 *
       7.7626) 62 77
tensor(0.) 71 71
tensor(1.00000e-02 *
       6.1899) 48 66
tensor(0.) 71 71
tensor(1.00000e-02 *
       7.1756) 62 77
tensor(0.) 71 71
tensor(0.1251) 30 59
tensor(0.) 71 71
tensor(1.00000e-02 *
       4.8789) 41 58
tensor(0.) 71 71
tensor(1.00000e-02 *
       6.3246) 48 66
tensor(0.) 71 71
tensor(1.00000e-02 *
       6.4264) 47 66
tensor(0.) 71 71
tensor(1.00000e-02 *
       5.8075) 48 66
tensor(0.) 71 71
tensor(1.00000e-02 *
       5.7100) 48 66
tensor(0.) 71 71
tensor(1.00000e-02 *
       7.7838) 47 66
tensor(0.) 71 71
tensor(1.00000e-02 *
       3.2765) 47 58
tensor(0.) 71 71
tensor(1.00000e-02 *
       5.9483) 48 66
tensor(0.) 71 71
tensor(1.00000e-02 *
       7.0395) 62 77
tensor(0.) 71 71
tensor(1.00000e-02 *
       5.9684) 47 66
tensor(0.) 71 71
tensor(1.00000e-02 *
       7.5795) 62 77
tensor(0.) 71 71
tensor(1.00000e-02

tensor(1.00000e-02 *
       3.2489) 48 66
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.1365) 51 66
tensor(0.) 71 71
tensor(1.00000e-02 *
       5.2156) 56 67
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.3870) 48 66
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.6050) 48 66
tensor(0.) 71 71
tensor(0.1079) 45 56
tensor(0.) 71 71
tensor(1.00000e-02 *
       1.7957) 51 66
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.2994) 51 66
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.9137) 48 66
tensor(0.) 71 71
tensor(1.00000e-02 *
       1.9667) 51 66
tensor(0.) 71 71
tensor(1.00000e-02 *
       1.8649) 51 66
tensor(0.) 71 71
tensor(1.00000e-02 *
       1.7263) 51 66
tensor(0.) 71 71
tensor(1.00000e-02 *
       2.8873) 48 66
tensor(0.) 71 71
tensor(0.1286) 52 77
tensor(0.) 71 71
tensor(1.00000e-03 *
       3.5943) 58 59
tensor(0.) 71 71
tensor(1.00000e-02 *
       1.6500) 53 60
tensor(0.) 71 71
tensor(0.1594) 45 73
tensor(0.) 71 71
tensor(1.00000e-02 *
       9.0829) 45 56
tensor(0.) 71 71
t